# setup

In [1]:
# !conda create -n py310 python=3.10 -y
# %pip install ipykernel
# !python -m ipykernel install --user --name=py310

In [4]:
# %pip install ipywidgets
# %pip install -r ../requirements.txt
# %pip install dataoceanai-dolphin

# create dataset

In [15]:
import os
import json
import torch
import torchaudio
from datasets import load_dataset

In [16]:
dataset = load_dataset("hsekhalilian/commonvoice", split="dev")
dataset = dataset.select(range(100))

In [17]:
output_dir = os.path.expanduser("~/.cache/datasets/commonvoice/audio_files/")
os.makedirs(output_dir, exist_ok=True)

output_manifest = os.path.join(output_dir, "..", "dev_manifest.json")
target_sr = 16000


with open(output_manifest, "w") as fout:
    for sample in dataset:
        text = sample["sentence"]
        
        audio_path = sample["audio"]["path"]
        audio_path = audio_path.strip("/").split("/")[-1]
        output_audio_path = os.path.join(output_dir, audio_path)
        
        waveform, sr = torch.tensor(sample["audio"]["array"]).unsqueeze(0), sample["audio"]["sampling_rate"]

        
        if sr != target_sr:
            resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=target_sr)
            waveform = resampler(waveform)
            sr = target_sr

        torchaudio.save(output_audio_path, waveform, sr)

        duration = waveform.shape[1] / sr

        fout.write(json.dumps({
            "audio_filepath": os.path.abspath(output_audio_path),
            "duration": duration,
            "text": text
        }) + "\n")

# inference

In [ ]:
import os
import json
import dolphin
from tqdm import tqdm
from jiwer import wer
from huggingface_hub import snapshot_download

In [41]:
model_name = "small"
model_dir = snapshot_download(repo_id=f"DataoceanAI/dolphin-{model_name}")

model = dolphin.load_model(model_name, model_dir, "cuda")

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

[2025-07-12 10:18:15,345] [INFO] [model.py:166:__init__] BatchBeamSearch implementation is selected.
[2025-07-12 10:18:15,351] [INFO] [model.py:177:__init__] Decoding device=cuda, dtype=float32
[2025-07-12 10:18:15,358] [INFO] [model.py:182:__init__] Text tokenizer: SentencepiecesTokenizer(model="/opt/conda/envs/py310/lib/python3.10/site-packages/dolphin/assets/bpe.model")


In [42]:
manifest_path = os.path.expanduser("~/.cache/datasets/commonvoice/dev_manifest.json")

audio_files = []
references = []
predictions = []

with open(manifest_path, "r", encoding="utf-8") as f:
    for line in f:
        entry = json.loads(line)
        audio_files.append(entry["audio_filepath"])
        references.append(entry["text"])

for audio_file in tqdm(audio_files):
    waveform = dolphin.load_audio(audio_file)
    result = model(waveform, lang_sym="fa", region_sym="IR")
    predictions.append(result.text_nospecial)
        
error_rate = wer(references, predictions)
print(f"\nWER: {error_rate:.2%}")

100%|██████████| 100/100 [00:42<00:00,  2.36it/s]


WER: 33.02%


In [44]:
for reference, prediction in zip(references, predictions):
    print(f"reference: {reference}")
    print(f"predicted: {prediction}")
    print("-"*30)

reference: این اولین قدم برای تغییر خودم
predicted: این اولین قدم برای تغییر خودم
------------------------------
reference: با خنده ای ترسناک چرا وحشت کردین؟ چرا تهمت می زنی؟
predicted: با خندهی ترسناک چرا وحشت کردین؟ چرا تهمت میزنین؟
------------------------------
reference: من همه جا دنبالت گشتم
predicted: من و همه جا دنبالت گشتم.
------------------------------
reference: افسانهها میگن سگها واسطهی دنیای زندهها با مردههان
predicted: افسانهها میگن سگها و ستار دنیای زندهها با مردهان
------------------------------
reference: فكر می کنم همین جا باید تمومش کنیم
predicted: فکر میکنم همینجا باید تمومش کنیم.
------------------------------
reference: اَفراسیاب
predicted: افرا سیاب
------------------------------
reference: طاهره چی بهش گفتی رنگش پرید
predicted: تا هرچی بهش گفتی رنگس برید؟
------------------------------
reference: من شبا خواب میبینم که سگها به هم حمله میکنن
predicted: من شبا خواب میبینم که سگا بهم حمله میکنی
------------------------------
reference: از وقتی که فقط پنج سالت بود و